In [ ]:
import os
import requests
import json
import sys
import os


##Create service accounts and tokens
#nucleus_uri = os.environ['DOMINO_API_HOST']



# What this notebook does

1. Create service account tokens (Run as an admin)
2. Make the service account a collaborator on a project (for convinience I just use the current project)
3. Attach a Github token to this service account profile
4. Attach that token to the project code for this service account
5. Run a trivial job using the service account token

# PRE-REQUSITES
You need two user environment variables to run the final section of this notebook
1. `SVC_ACCOUNT_TOKEN` - This is the service account token you generate
2. `SVC_GITHUB_TOKEN` - This is the github token you use to attach to the SVC_Account and eventually map to the project for that service account.

Step (2) is what allows you to run jobs using the SVC_ACCOUNT_TOKEN inside a Git-backed project

## Why Service Account?

Service Accounts are industry standard ways of creating identities which only interact with the platform programmatically. They are identities used to execute automation workflows.


## What functions does this notebook expose?

1. List all service accounts - `get_all_service_accounts`
2. Create a new service account - `create_sa(sa_name)`
3. Get the Keycloak Id associated with each SA - `get_idp_id_by_sa()`
4. Get the Domino (Mongo) Id associated with each SA - `get_id_by_sa()`
5. Create a named token for a service account. A service account can have many named tokens -`create_token(idpid,token_name)`
6. Invalidate token for a service account. A token that is invalidated cannot be reactivated - `invalidate_token(token_name):`
7. Delete token for a service account. A token with the same name can be recreated after its deleted - `delete_token(token_name)`
8. Add a service account as a project collaborator (collab_id==sa_mongo_id) - `add_sa_as_project_collaborator(project_id,collab_id,project_role)`
9. Get git credential id's associated with all users (id==mongo_id)- `get_git_credentials(id)`
10. Get repo_id associated with a git backed project - `get_project_repo_id(project_id)`
11. Get git credentials associated with a give project - `get_git_credentials`
12. Add a git credential for a collaborating project for a given sa `add_git_cred_for_project_for_sa`
13. Delete git credentials for a collaborating project for a given sa `delete_git_credentials_for_sa`

(11,12,13) in the real world would be run by a SA token after the SA is added as a collaborator for a give project




In [ ]:
api_proxy = os.environ['DOMINO_API_PROXY']
svc_accounts_endpoint= f'{api_proxy}/v4/serviceAccounts'
## First step is fetch all service account
def  get_all_service_accounts():
    print(svc_accounts_endpoint)
    resp = requests.get(svc_accounts_endpoint)
    print(resp)
    users=[]
    if resp.status_code==200:
        for u in resp.json():
            users.append(u['username'])
    return users

def create_sa(sa_name:str):
    data = {'username':sa_name,'email':f'{sa_name}@xyz.com'}
    resp = requests.post(svc_accounts_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(f'Response for creating svc account {sa_name} is {resp.status_code}')
    print(resp.json())

#Now get Idp Id of all emails. This only returns active service accounts. If you have deactivated the SA
#you can no longer see it. The only place to get that is from Mongo or Keycloak to reactivate
#Activation of SA using names is not supported
def get_idp_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['idpId']
    return idp_by_user
def get_id_by_sa():
    resp = requests.get(svc_accounts_endpoint)
    print(resp.json())
    idp_by_user={}
    if resp.status_code==200:
        for u in resp.json():
            idp_by_user[u['username']]=u['id']
    return idp_by_user
# You cannot overwrite an existing or even an invalidated token. You have to delete it first
def create_token(idpid,token_name):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    resp = requests.post(f"{token_endpoint}",json={"name":token_name},
                         headers={'Content-Type':'application/json'})
    print(resp.status_code)
    return resp.json()

def deactivate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/deactivate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Deactivating user {user}")
def activate_sa(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/activate'        
    resp = requests.post(token_endpoint,headers={'Content-Type':'application/json'})
    print(f"Status code {resp.status_code} - Activating user {user}")

#List all tokens
def list_all_tokens(idpid):
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens'    
    
    resp = requests.get(token_endpoint,headers={'Content-Type':'application/json'})
    
    if resp.status_code==200:
       out = resp.json()           
       pretty_json = json.dumps(out, indent=4)
       print(pretty_json)

def invalidate_token(token_name):
    print(f"Invalidating token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.post(f"{token_endpoint}/invalidate",headers={'Content-Type':'application/json'})
    print(resp.status_code)
def delete_token(token_name):
    print(f"Deleting token {token_name}?")
    token_endpoint= f'{svc_accounts_endpoint}/{idpid}/tokens/{token_name}'    
    resp = requests.delete(f"{token_endpoint}",headers={'Content-Type':'application/json'})
    print(resp.status_code)
    

In [ ]:
print(get_all_service_accounts())

In [ ]:
## Create Service Accounts if they don't exist
svc_accounts=['svc-user-ds-u1','svc-admin-u2','svc-user-ds-u3']
for sa_name in svc_accounts:
    ## 200 
    create_sa(sa_name)

In [ ]:
idp_by_user = get_idp_id_by_sa()

In [ ]:
## Create tokens for SA. Will return status code 200 if succeeded or 500 if it already exists
results = []
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"    
    j = create_token(idpid,token_name)
    results.append(j)

with open("/tmp/tokens.json", "w") as file:
    json.dump(results, file, indent=2)

In [ ]:
## Activate/Deactivate SVC Tokens. Do not use it currently because a token created for a deactivated/activated user
## does not have `sub` field or claims making it unsuable
'''
for user, idpid in idp_by_user.items():
    deactivate_sa(idpid)
for user, idpid in idp_by_user.items():
    activate_sa(idpid)
'''

In [ ]:
for user, idpid in idp_by_user.items():
    list_all_tokens(idpid)

In [ ]:
## Invalidate and delete tokens examples
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # Invalidated tokens cannot be used or be revalidated
    invalidate_token(token_name)

for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"
    # You have to delete a token to recreate it
    delete_token(token_name)

In [ ]:
#Now recreate them
## Create tokens for SA. Will return status code 200 if succeeded or 500 if it already exists
results = []
for user, idpid in idp_by_user.items():    
    token_name = f"{user}-token-app"    
    j = create_token(idpid,token_name)
    results.append(j)

with open("/tmp/tokens.json", "w") as file:
    json.dump(results, file, indent=2)

In [ ]:

def add_sa_as_project_collaborator(project_id,collab_id,project_role):
    project_endpoint= f'{api_proxy}/v4/projects/{project_id}/collaborators'
    data = {"collaboratorId":collab_id,
            "projectRole":project_role}
    resp = requests.post(url=project_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(resp.status_code)
    print(resp.json())
    

In [ ]:
## As an admin add the SA to the project as a collaborator. This is needed to allow them to run jobs
project_id = os.environ['DOMINO_PROJECT_ID']
id_by_user = get_id_by_sa()
for u,id in id_by_user.items():
    add_sa_as_project_collaborator(project_id,id,"Contributor")


In [ ]:
## Now we need to add a git credential to a service account

In [ ]:
def get_git_credentials(id,svc_token):  
    api_proxy = os.environ['DOMINO_API_PROXY']
    headers = {'Content-Type':'application/json'}
    
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials"    
    resp = requests.get(url,headers=headers)   
    print(resp.status_code)
    print(resp.text)
    return resp.json()

def delete_git_credentials_for_sa(id,credential_id,svc_token):    
    #headers = {'Content-Type':'application/json'}
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    url = f"{api_proxy}/v4/accounts/{id}/gitcredentials/{credential_id}"    
    resp = requests.delete(url,headers=headers)   
    print(resp.status_code)
    return resp.json()

def add_git_credentials_to_sa(domino_id,git_token_name,git_token,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    #api_proxy = os.environ['DOMINO_API_PROXY']
    #headers = {'Content-Type':'application/json'}

    url = f"{api_proxy}/v4/accounts/{domino_id}/gitcredentials"
    p = {"name":git_token_name,"gitServiceProvider":"github","accessType":"token", "token":git_token,"type":"TokenGitCredentialDto"}
    resp = requests.post(url,json=p,headers=headers)   
    print(resp.status_code)
    return resp.json()



In [ ]:
def get_project_repo_id(project_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    
    url = f"{api_proxy}/v4/projects/{project_id}"    
    obj = requests.get(url,headers=headers).json()
    print(obj)
    git_repository_id = obj['mainRepository']['id']
    return git_repository_id

##Run this using the service account token. This is for illustration only
def add_git_cred_for_project_for_sa(project_id,repo_id,cred_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_host}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.put(url,json={"credentialId": cred_id},headers=headers)
    print(out.status_code)
    print(out.text)

def get_git_cred_for_project_for_sa(project_id,repo_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}
    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.get(url,headers=headers)
    print(out.status_code)
    print(out.text)

def delete_git_cred_for_project_for_sa(project_id,repo_id,svc_token):
    api_host = os.environ['DOMINO_API_HOST']
    headers = {'Content-Type':'application/json', 'Authorization': f"Bearer {svc_token}"}

    url = f"{api_proxy}/v4/projects/{project_id}/repository/{repo_id}/credentialMapping"
    out = requests.delete(url,headers=headers)
    print(out.status_code)
    print(out.text)


In [ ]:
import base64
import json

def decode_jwt(token: str):
    header_b64, payload_b64, signature_b64 = token.split('.')

    # Base64URL decoding requires padding correction
    def b64url_decode(b64_str):
        padding = '=' * (-len(b64_str) % 4)
        return base64.urlsafe_b64decode(b64_str + padding)

    header = json.loads(b64url_decode(header_b64))
    payload = json.loads(b64url_decode(payload_b64))
    
    return {
        "header": header,
        "payload": payload,
        "signature": signature_b64  # raw, usually verified separately
    }

# Example usage
#decoded = decode_jwt(svc_token)
#print(json.dumps(decoded, indent=2))

In [ ]:
svc_token=os.environ['SVC_ACCOUNT_TOKEN']
#svc_token="eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI2ekZfWlgzRlJiV0dnT215VHJKd0xmXzFVNUxUeENVT0t4bTNvSDhZNW5ZIn0.eyJleHAiOjE3Njc0NTc2OTgsImlhdCI6MTc1NzA4OTY5OCwianRpIjoiYzBhODc1ZmItMTlhZC00ZmVlLWI4NTgtZjU3MGQ3ZWVmYzUxIiwiaXNzIjoiaHR0cHM6Ly9tYXJjZG83NzM2NC5jcy5kb21pbm8udGVjaC9hdXRoL3JlYWxtcy9Eb21pbm9SZWFsbSIsImF1ZCI6WyJkb21pbm8tcGxhdGZvcm0iLCJhY2NvdW50Il0sInN1YiI6IjY4YjllZmJjZWYwZjJkMWYxZTk1ZmRlNCIsInR5cCI6IkJlYXJlciIsImF6cCI6ImRvbWluby1zZXJ2aWNlLWFjY291bnRzIiwic2lkIjoiNjdiZThhZDMtMDA1NS00YzM4LWFiYTUtMGViOTAyZGNiYWIwIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLWRvbWlub3JlYWxtIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiZG9taW5vLXNlcnZpY2UtYWNjb3VudHMiOnsicm9sZXMiOlsiZG9taW5vLXNlcnZpY2UtYWNjb3VudCJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZG9taW5vLWp3dC1jbGFpbXMgZW1haWwgb2ZmbGluZV9hY2Nlc3MgcHJvZmlsZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiaWRwX2lkIjoiZDA5YTk0OTctMGEwNy00YmI4LThkMzAtNDQ3N2MwZTZjNGU1IiwibmFtZSI6IlNBIHN2Yy11c2VyLWRzLXUzIiwicHJlZmVycmVkX3VzZXJuYW1lIjoic3ZjLXVzZXItZHMtdTMiLCJnaXZlbl9uYW1lIjoiU0EiLCJmYW1pbHlfbmFtZSI6InN2Yy11c2VyLWRzLXUzIiwiZW1haWwiOiJzdmMtdXNlci1kcy11M0B4eXouY29tIiwidXNlcl9ncm91cHMiOlsiL3JvbGVzL1ByYWN0aXRpb25lciJdfQ.vINxSkFFVOP8WbP4NEhtEPMHAHu8DTaSx1_GuXRSzSKDvUOYuX6oVQXNHJyXVKH_Wo098l-49sk3MdcQC_4JTe-yptl3WoBkP7s1Fo5AkavWGAAUFkuk2nL1zSef4V51pZ_Gu4aaVP2GuwYeSTiNWBLlYnQ7NUeKJADW4HgSTH-_9IcvH8vCt2H-X7NHmLWD-PpuFH92pEHIOrm-WtTtJke5YKRtf0Rc8CuM2CXdL7KLfRvfLCegZzpIANWEnly75-uNA3aOlonjD9WifXGIwldAh0DszLxZ2DDribHTO8ePIhtvXnX5HRdYh84vX76VwEpjsritvQz3QFEadaQuvA"
svc_jwt = decode_jwt(svc_token)
#Sanity check. If your svc token cannot resolve this, it won't work
svc_account_name = svc_jwt['payload']['name']
idp_id = svc_jwt['payload']['idp_id']
domino_id = svc_jwt['payload']['sub']
print(f"SVC_NAME={svc_account_name}, IDP_NAME={idp_id}, SUB={domino_id}")

In [ ]:
#Let is find the git credential associated with this service account
git_token_name = "test-git-token"
creds = get_git_credentials(domino_id,svc_token)
print(creds)
#Optionally delete them before re-adding them
for c in creds:    
    credential_id = c['id']
    delete_git_credentials_for_sa(id,credential_id,svc_token)

#Now let us add the git credentials
print(add_git_credentials_to_sa(domino_id,git_token_name,os.environ['SVC_GITHUB_TOKEN'],svc_token))


In [ ]:
#Also add it for yourself just to make sure you know it works for you. And this allow you to test this token from the UI before asking the service account to start a job
my_token = requests.get(f"{api_proxy}/access-token").text 
my_domino_id = decode_jwt(my_token)['payload']['sub']
print(add_git_credentials_to_sa(my_domino_id,git_token_name,os.environ['SVC_GITHUB_TOKEN'],my_token))

In [ ]:
my_token = decode_jwt(requests.get(f"{api_proxy}/access-token").text)
my_domino_id = my_token['payload']['sub']
print(my_domino_id)

In [ ]:
project_id = os.environ['DOMINO_PROJECT_ID']
repo_id = get_project_repo_id(project_id,svc_token)
print(f"Repo id {repo_id}")

In [ ]:
##This should be run using each service account tokens 
#for u,id in id_by_user.items():


creds = get_git_credentials(domino_id,svc_token)
print(creds)
credential_id = creds[0]['id']
print("Get Git Cred mapping")
print(get_git_cred_for_project_for_sa(project_id,repo_id,svc_token))

print("Delete Git Cred mapping")
delete_git_cred_for_project_for_sa(project_id,repo_id,svc_token)

print("Get Git Cred mapping")
print(get_git_cred_for_project_for_sa(project_id,repo_id,svc_token))


print("Add Git Cred mapping")
add_git_cred_for_project_for_sa(project_id,repo_id,credential_id,svc_token)

In [ ]:
## And now you can use the service account token to run jobs for a git backed project

In [ ]:
import os
import requests

def get_hw_tier_id(name: str):
    api_proxy = os.environ['DOMINO_API_PROXY']
    url = f"{api_proxy}/v4/hardwareTier"
    results = requests.get(url)
    hw_tiers = results.json()['hardwareTiers']
    for h in hw_tiers:
        if h['name']==name:
            return h

def get_environment_id(name: str):
    api_proxy = os.environ['DOMINO_API_PROXY']
    url = f"{api_proxy}/v4/environments/self"
    results = requests.get(url)
    envs = results.json()
    for e in envs:
        if e['name']==name:
            return e


In [ ]:

from domino import Domino
import os
import requests

svc_token=os.environ['SVC_ACCOUNT_TOKEN']
domino = Domino("wadkars/ddl-end-to-end-demo",auth_token=svc_token)
compute_env_id = get_environment_id('Domino Standard Environment Py3.10 R4.4')['id']
title = f"Test Job for SVC Account"
j = domino.job_start(title=title,command="/tmp/ls",
                 hardware_tier_name="Small",environment_id=compute_env_id)
print(j)